In [ ]:
#AI Trainer for the Adjustments Project


In [13]:
#Load input data
import pandas as pd
import numpy as np

#Load data
df=pd.read_csv('SampleData.csv')
df.head(10)

,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,OldVal,NewVal
0,Amend,GRC_EndMonth,CountryCode,616523,474892,0.58,CUST563201,Corporate,Low,1,DE,DE,DE
1,Amend,GRC_EndMonth,RiskCategory,30867,968743,0.92,CUST359147,Retail,Low,1,GB,Low,High
2,Amend,Credit_Uplift,CustomerType,673759,735775,0.68,CUST238947,SME,Medium,1,US,SME,Retail
3,Amend,Default,CustomerType,427442,928956,0.73,CUST103289,Corporate,Low,1,US,Corporate,Corporate
4,Amend,GRC_EndMonth,CustomerType,257396,354727,0.09,CUST563201,Corporate,High,0,GB,Corporate,Retail
5,Amend,GRC_EndMonth,CountryCode,504669,250845,0.98,CUST450896,Corporate,High,1,DE,DE,US
6,Amend,Default,IsDefaulted,243263,204382,0.23,CUST981372,Retail,Low,0,US,0,0
7,Amend,GRC_EndMonth,CustomerType,907153,663508,0.55,CUST804231,Corporate,Low,1,GB,Corporate,SME
8,Amend,Credit_Uplift,CustomerID,983692,53581,0.66,CUST712035,Retail,Medium,0,GB,CUST712035,CUST981372
9,Amend,GRC_EndMonth,CountryCode,601407,241378,0.36,CUST665428,Corporate,Low,0,US,US,DE


In [ ]:
#Config for the preprocessing


In [16]:
#Stage 1: OneHot encode string labels so we can use them in the model

#Build list of all string columns in the dataframe
string_columns=[]
for col in df.columns:
    if df[col].dtype=='object':
        string_columns.append(col)


df_encoded=pd.get_dummies(df, columns=string_columns)
df_encoded


,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,NewVal_CUST981372,NewVal_Corporate,NewVal_DE,NewVal_GB,NewVal_High,NewVal_Low,NewVal_Medium,NewVal_Retail,NewVal_SME,NewVal_US
0,616523,474892,0.58,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,30867,968743,0.92,1,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,673759,735775,0.68,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,427442,928956,0.73,1,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,257396,354727,0.09,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,943881,11157,0.16,1,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
996,229135,345821,0.38,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
997,143371,868564,0.90,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
998,830759,239910,0.75,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
#Stage 2: Normalize all numeric values
#Normalize the dataframe
df_normalized = (df_encoded - df_encoded.mean()) / df_encoded.std()
df_normalized

,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,NewVal_CUST981372,NewVal_Corporate,NewVal_DE,NewVal_GB,NewVal_High,NewVal_Low,NewVal_Medium,NewVal_Retail,NewVal_SME,NewVal_US
0,0.417507,-0.119279,0.311414,0.991536,NaN,-0.509104,-0.488793,-0.513769,-0.501311,2.057029,...,-0.131441,-0.265694,3.729805,-0.243439,-0.282549,-0.259169,-0.256967,-0.248011,-0.286651,-0.286651
1,-1.598145,1.566381,1.514604,0.991536,NaN,-0.509104,-0.488793,-0.513769,-0.501311,2.057029,...,-0.131441,-0.265694,-0.267842,-0.243439,3.535675,-0.259169,-0.256967,-0.248011,-0.286651,-0.286651
2,0.614496,0.771192,0.665293,0.991536,NaN,1.962273,-0.488793,-0.513769,-0.501311,-0.485652,...,-0.131441,-0.265694,-0.267842,-0.243439,-0.282549,-0.259169,-0.256967,4.028044,-0.286651,-0.286651
3,-0.233253,1.430576,0.842233,0.991536,NaN,-0.509104,2.043812,-0.513769,-0.501311,-0.485652,...,-0.131441,3.759969,-0.267842,-0.243439,-0.282549,-0.259169,-0.256967,-0.248011,-0.286651,-0.286651
4,-0.818500,-0.529438,-1.422595,-1.007528,NaN,-0.509104,-0.488793,-0.513769,-0.501311,2.057029,...,-0.131441,-0.265694,-0.267842,-0.243439,-0.282549,-0.259169,-0.256967,4.028044,-0.286651,-0.286651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.544175,-1.702145,-1.174880,0.991536,NaN,-0.509104,-0.488793,-0.513769,-0.501311,2.057029,...,-0.131441,-0.265694,3.729805,-0.243439,-0.282549,-0.259169,-0.256967,-0.248011,-0.286651,-0.286651
996,-0.915766,-0.559837,-0.396345,0.991536,NaN,-0.509104,-0.488793,-0.513769,-0.501311,2.057029,...,-0.131441,-0.265694,-0.267842,-0.243439,-0.282549,-0.259169,-0.256967,-0.248011,-0.286651,-0.286651
997,-1.210940,1.224440,1.443828,-1.007528,NaN,-0.509104,2.043812,-0.513769,-0.501311,-0.485652,...,-0.131441,-0.265694,-0.267842,-0.243439,3.535675,-0.259169,-0.256967,-0.248011,-0.286651,-0.286651
998,1.154843,-0.921343,0.913009,-1.007528,NaN,-0.509104,-0.488793,-0.513769,1.992774,-0.485652,...,-0.131441,-0.265694,-0.267842,-0.243439,-0.282549,-0.259169,-0.256967,-0.248011,-0.286651,3.485073
